### Préparation des données

- merged4 + colomne 3 = 99% acc
- merged3 + colomne 3 = 81% acc
- merged2 + colomne 4 = 94% acc
- merged5 + colomne 2 = 84% acc
- merged1 + colomne 1 = 53% acc

In [9]:
import sys
sys.path.append('../src/')

import dataset_prep

#liste des path :
# "../data/1416/merged1.csv"
# "../data/1416/merged2.csv"
# "../data/1416/merged3.csv"
# "../data/1416/merged4.csv"
# "../data/1416/merged5.csv"
# "../data/1416/data1416.csv"
# "../data/0709/merged1.csv"
dataset_path = "../../data/0709/merged1.csv"


# liste des colomnes
# receiver, sender, sender pseudo, message Id, scenario , a supprimé
# merged1 : "sendTime", "rcvTime", "pos_y_rec_f", "pos_y_rec", "pos_x_rec_f", "pos_x_rec", "pos_y_send_f", "pos_y_send", "pos_x_send_f", "pos_x_send", "label"
# merged2 : "spd_x_send", "max_speed2", "max_speed1", "distance", "distRealSR2", "distRealSR1", "diffSpdSR2", "diffSpdSR1", "difSpeed", "deltaSpd2", "label"
# merged3 : "sendTime", "rcvTime", "pos_x_rec_f", "max_speed1", "distRealSR2", "distRealSR1", "deltaSpd2", "deltaSpd1", "deltaPos2", "deltaPos1", "label"
# merged4 : "pos_x_rec_f", "nb_packets_sent", "distance", "distRealSR2", "distRealSR1", "deltaPos2", "deltaPos1", "spd_y_send", "spd_x_send", "sendTime", "label"
# merged5 : "pos_x_rec_f", "pos_x_rec", "nb_packets_sent", "distance", "distRealSR2", "distRealSR1", "deltaPos2", "deltaPos1", "sendTime", "rcvTime", "label"
# 0709 : "time_diff", "nb_packets_sent", "max_speed1", "max_speed2", "hed_noise_x", "sendTime", "rcvTime", "hed_noise_y", "label"
usecols = ["time_diff", "nb_packets_sent", "max_speed1", "max_speed2", "hed_noise_x", "sendTime", "rcvTime", "hed_noise_y", "label"]

data = dataset_prep.import_dataset(dataset_path, usecols)
dataset_prep.clean_dataset(data)
X_train, X_test, y_train, y_test = dataset_prep.data_preparation(data, test_size=0.1)


Nombre de lignes avant nettoyage :  1048575
Nombre de lignes après nettoyage :  1013977


# Classification par clustering k-means
https://realpython.com/k-means-clustering-python/

In [10]:
import matplotlib.pyplot as plt
from kneed import KneeLocator
from sklearn.datasets import make_blobs
from sklearn.cluster import KMeans, DBSCAN
from sklearn.metrics import silhouette_score, classification_report, adjusted_rand_score
from sklearn.preprocessing import StandardScaler

### Standardisation des données

Centrée et normalisé les données :  moyenne de 0 et standard déviation de 1

In [286]:
scaler = StandardScaler()
scaled_X_train = scaler.fit_transform(X_train)
scaled_X_test = scaler.fit_transform(X_test)

y_train = [0 if y_train[i]==0 else 1 for i in range(len(y_train))]
y_test = [0 if y_test[i]==0 else 1 for i in range(len(y_test))]


### Préparation et entraiement du modèle kmeans

In [287]:
kmeans = KMeans(init="random", n_clusters=2, n_init=100, max_iter=300)
kmeans.fit(scaled_X_train, y_train)

KMeans(init='random', n_clusters=2, n_init=100)

In [288]:
print("The lowest SSE (sum of the squared error) value",kmeans.inertia_)
print("The number of iterations required to converge", kmeans.n_iter_)
#print("Predict labels", kmeans.labels_, "VS True label", y_train)
print("Number of features seen during fit", kmeans.n_features_in_)

The lowest SSE (sum of the squared error) value 5878581.163780677
The number of iterations required to converge 6
Number of features seen during fit 8


In [289]:
y_pred=kmeans.predict(scaled_X_test)
print("predicted label", y_pred)
#print("True label", y_test)

print(classification_report(y_test, y_pred))

# Adjusted rand index (ARI)
ari_kmeans = adjusted_rand_score(y_train, kmeans.labels_)

print("ARI", ari_kmeans)

predicted label [0 0 0 ... 1 0 1]
              precision    recall  f1-score   support

           0       0.48      0.45      0.46     44608
           1       0.59      0.61      0.60     56790

    accuracy                           0.54    101398
   macro avg       0.53      0.53      0.53    101398
weighted avg       0.54      0.54      0.54    101398

ARI 0.006202175616325859
